In [ ]:
# === Financial GenAI Recommender PoC ===
# Author: ChatGPT + User Collaboration
# Description: Full PoC pipeline for personalized financial recommendations using text-based user input.
# Covers: Sentiment Analysis, Intent Detection, Keyword Extraction, Rule-Based Recommendation, Gradio UI

# === Assumptions ===
# 1. No internet access: All models must run locally (TextBlob, sklearn, etc.)
# 2. No OpenAI API or HuggingFace API used (but prompt templates are LLM-friendly)
# 3. Dataset: We'll use a small simulated list of user messages as the input
# 4. Model training is skipped in favor of prompt-style logic and local ML components
# 5. Product/Content recommendations are mocked via keyword/intent mapping

# === Requirements ===
# pip install gradio textblob scikit-learn pandas
# python -m textblob.download_corpora

import gradio as gr
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re

# === Simulated User Messages Dataset ===
user_posts = [
    "Thinking of switching to low-risk investments after this crypto dip",
    "Booked a few luxury hotels recently, maybe it's time for a better credit card",
    "Looking to learn more about mutual funds before I invest",
    "The market is crashing again. Should I sell my tech stocks?",
    "Just paid off my loan early! Now what?",
    "Interest rates are insane lately, I'm done with my savings account",
    "Hearing about safe ETFs – what's the best option for someone like me?"
]

# === Product Mapping Based on Intent + Sentiment ===
RECOMMENDATION_MAP = {
    ("Investing", "Positive"): ["Growth Mutual Funds", "ETF Portfolio"],
    ("Investing", "Negative"): ["Low-Risk Bonds", "Liquid Funds"],
    ("Learning", "Neutral"): ["Intro to Stocks Guide", "Mutual Fund Crash Course"],
    ("Credit", "Positive"): ["Premium Credit Card", "Travel Reward Card"],
    ("Credit", "Negative"): ["No Annual Fee Card", "Prepaid Card Options"],
    ("Exit", "Negative"): ["Risk-Hedged Portfolio", "Capital Protection Plans"],
    ("Savings", "Negative"): ["High Yield Accounts", "Digital Gold"]
}

# === Intent Extraction Logic ===
def extract_intents(text):
    intents = []
    t = text.lower()
    if any(x in t for x in ["buy", "invest", "etf", "portfolio"]): intents.append("Investing")
    if any(x in t for x in ["learn", "understand", "study"]): intents.append("Learning")
    if any(x in t for x in ["loan", "credit card", "emi", "spend"]): intents.append("Credit")
    if any(x in t for x in ["sell", "exit", "crash"]): intents.append("Exit")
    if any(x in t for x in ["save", "interest", "fixed deposit"]): intents.append("Savings")
    if not intents:
        intents.append("General")
    return intents[:3]  # top 3

# === Sentiment Analysis ===
def get_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"

# === Keyword Extraction ===
def extract_keywords(text, top_k=5):
    tfidf = TfidfVectorizer(stop_words='english')
    X = tfidf.fit_transform([text])
    indices = X[0].toarray().argsort()[0][-top_k:][::-1]
    return [tfidf.get_feature_names_out()[i] for i in indices]

# === Recommendation Engine ===
def recommend_products(intents, sentiment):
    results = []
    for intent in intents:
        match = RECOMMENDATION_MAP.get((intent, sentiment))
        if match:
            results.extend(match)
    return list(set(results)) if results else ["We recommend speaking with a financial advisor for personalized help."]

# === Combined GenAI Engine ===
def financial_assistant(text):
    sentiment = get_sentiment(text)
    intents = extract_intents(text)
    keywords = extract_keywords(text)
    recommendations = recommend_products(intents, sentiment)

    return {
        "Sentiment": sentiment,
        "Intents": intents,
        "Keywords": keywords,
        "Recommended Products/Content": recommendations
    }

# === Gradio Interface ===
with gr.Blocks(title="🧠 Financial GenAI Assistant") as demo:
    gr.Markdown("""# 🤖 Financial Recommender (PoC)
Paste a financial thought or comment and we'll generate sentiment, intent, keywords, and personalized suggestions!
""")
    inp = gr.Textbox(label="💬 Your Financial Thought", placeholder="e.g. I might sell my crypto after this dip.")
    btn = gr.Button("Generate Recommendation")
    out = gr.JSON(label="🧠 AI Analysis")

    btn.click(fn=financial_assistant, inputs=inp, outputs=out)

if __name__ == "__main__":
    demo.launch()
